# Data preperation notebook

In [1]:
import requests
import time
import pandas as pd

# Define the endpoint and parameters
base_url = "https://api-testnet.bybit.com"
endpoint = "/v5/market/kline"
symbol = "BTCUSD"  # Example trading pair
interval = "1"    # Kline interval, e.g., "1", "5", "15", "60", "240", etc.
limit = 4        # Number of klines to fetch
start_time = int(time.time()) - 3600 * 24  # Start time in seconds (e.g., 24 hours ago)

# Construct the URL
url = f"{base_url}{endpoint}?category=linear&symbol={symbol}&interval={interval}&limit={limit}"#&start_time={start_time}"

# Make the GET request
response = requests.get(url,headers={}, data={})
data = response.json() 

print(data)
print(pd.Timestamp(data["time"],unit="ms"))
print(data["result"]["list"])



{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSD', 'category': 'linear', 'list': [['1716562500000', '67863.5', '67884.5', '67863.5', '67884.5', '685944', '10.10657531'], ['1716562440000', '67794', '67915', '67790.5', '67863.5', '101489', '1.49547534'], ['1716562380000', '67778', '67794', '67776.5', '67794', '165', '0.00243425'], ['1716562320000', '67757', '67778', '67757', '67778', '115', '0.0016968']]}, 'retExtInfo': {}, 'time': 1716562539615}
2024-05-24 14:55:39.615000
[['1716562500000', '67863.5', '67884.5', '67863.5', '67884.5', '685944', '10.10657531'], ['1716562440000', '67794', '67915', '67790.5', '67863.5', '101489', '1.49547534'], ['1716562380000', '67778', '67794', '67776.5', '67794', '165', '0.00243425'], ['1716562320000', '67757', '67778', '67757', '67778', '115', '0.0016968']]


In [2]:
#def prepare_data(data):
print(pd.to_datetime(int(data["result"]["list"][0][0]),unit='ms'))
print(pd.to_datetime(int(data["result"]["list"][1][0]),unit='ms'))
print(pd.to_datetime(int(data["result"]["list"][2][0]),unit='ms'))
print(pd.to_datetime(int(data["result"]["list"][3][0]),unit='ms'))



2024-05-24 14:55:00
2024-05-24 14:54:00
2024-05-24 14:53:00
2024-05-24 14:52:00


In [3]:
import pandas as pd
import numpy as np

def prepare_data(data):

    # define pandas DF 
    cols = ['time','open', 'high', 'low', 'close','vol','vol_coin']
    df = pd.DataFrame(data["result"]["list"], columns=cols)

    df['time'] = pd.Timestamp(df["time"],unit="ms"))
    df["day"] = df["time"].dt.day
    df["hour"] = df["time"].dt.hour


    # applying logs
    for col in cols:
        df["log_"+str(col)] = np.log(df[str(col)].astype('float64')+1)

    # >>> AS FOR NOW useless
    # normalize along "MinMaxScaler" (same like sci kit learn)
    # for col in cols+["log_vol"]:
    #    df["norm_"+str(col)] =  (df[str(col)].astype('float64') - np.min(df[str(col)].astype('float64')) ) / ( np.max(df[str(col)].astype('float64')) - np.min(df[str(col)].astype('float64')) ) 

    # Stack the DataFrame
    stacked_df = df.stack()
    
    # Create a new DataFrame from the stacked series and transpose it
    # this creates a multi index data frame with tuples as indices, like [(0,'ts'),...]
    df_single_row = stacked_df.to_frame().T
    
    # now change the multiindex col to a single index col by replacing it witht a list of concatenated strings 
    df_single_row.columns = [f'{col[1]}_{col[0]}' for col in df_single_row.columns]
    
    return df_single_row



prep_df = prepare_data(data)
prep_df

,ts_0,open_0,high_0,low_0,close_0,vol_0,vol_coin_0,log_ts_0,log_open_0,log_high_0,...,close_3,vol_3,vol_coin_3,log_ts_3,log_open_3,log_high_3,log_low_3,log_close_3,log_vol_3,log_vol_coin_3
0,1716562500000,67863.5,67884.5,67863.5,67884.5,685944,10.10657531,28.171345,11.125268,11.125578,...,67778,115,0.0016968,28.171345,11.123698,11.124008,11.123698,11.124008,4.75359,0.001695


# Creating a database

In [4]:
from sqlalchemy import create_engine
#engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

engine = create_engine('sqlite:///BTCUSDTstream-test.db')

In [5]:
# write function to transform data from bybit call    

In [6]:
prep_df.to_sql('BTCUSDT',engine, if_exists = 'append', index=False)

1

In [7]:
pd.read_sql('BTCUSDT',engine)

,ts_0,open_0,high_0,low_0,close_0,vol_0,vol_coin_0,log_ts_0,log_open_0,log_high_0,...,close_3,vol_3,vol_coin_3,log_ts_3,log_open_3,log_high_3,log_low_3,log_close_3,log_vol_3,log_vol_coin_3
0,1716284040000,71056.5,71063,71053,71056.5,200082,2.81566749,28.171183,11.171245,11.171336,...,71037,13,0.00018291,28.171183,11.170815,11.170970,11.170815,11.170970,2.639057,0.000183
1,1716562500000,67863.5,67884.5,67863.5,67884.5,685944,10.10657531,28.171345,11.125268,11.125578,...,67778,115,0.0016968,28.171345,11.123698,11.124008,11.123698,11.124008,4.753590,0.001695


In essence the function calls would look like this: 

response = fetch_kline()
pred_df = prepare_data(response)

entry,stop_loss,take_profit = make_prediction(prep_df) # add column if trade or not 

write_db(prep_df)

place_order(entry,stop_loss,take_profit)

